In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import jaccard_score

from skimage import filters

from medpy.filter import smoothing

import generator
import data_manager as dm
import pixel_counter as pc
from helper import time_measurement

In [ ]:
def preview_volume(volume, axis):
    dim = len(volume.shape)
    if 2 > dim > 3:
        raise ValueError('volume should be 2D or 3D')
    image = volume[:, :, 0] if dim == 3 else volume
    axis.imshow(image, cmap='gray')


def preview_volumes(volume0, volume1, title):
    fig, axes = plt.subplots(1, 2, figsize=(20, 10))
    fig.suptitle(title)
    preview_volume(volume0, axes[0])
    preview_volume(volume1, axes[1])


def show_phantoms_stats(phantom, processed_phantom, title):
    
    porous = (phantom == 1).sum()
    stones = (phantom == 0).sum()
    
    print(f'porosity: { porous / (porous + stones) }\n')
    
    ph = np.ravel(phantom)
    p_ph = np.ravel(processed_phantom)
    p_ph_porous = p_ph[ph == 1]
    p_ph_stones = p_ph[ph == 0]
    
    plt.figure(figsize=(20, 10))
    plt.title(title)
    plt.hist(p_ph, 255, [0, 1], color='lightgray')
    plt.hist(p_ph_porous, 255, [0, 1], histtype='step', color='red')
    plt.hist(p_ph_stones, 255, [0, 1], histtype='step', color='blue')

In [ ]:
size = 100
dim = 3
shape = tuple(size for _ in range(dim))
porosity = 0.5
blobiness = 1
noise_method = 'poisson'
noise = 3e2
num_of_angles = 90
number_of_trains = 2

In [ ]:
def generate_phantoms():

    for i in range(number_of_trains):
        print(f'train { i }:')
        generator.create_phantom_and_process(
            shape, porosity, blobiness, noise, num_of_angles, 'train', noise_method=noise_method
        )
    
    print('test:')
    phantom, processed_phantom = generator.create_phantom_and_process(
        shape, porosity, blobiness, noise, num_of_angles, 'test', noise_method=noise_method
    )
    title = f'porosity: { porosity }, blobiness: { blobiness }, noise: { noise }'
    preview_volumes(phantom, processed_phantom, title)
    show_phantoms_stats(phantom, processed_phantom, title)

In [ ]:
generate_phantoms()

In [ ]:
data_info = dm.show_data_info()
data_info

In [ ]:
im_lenght = np.prod(shape)
coeff = 100
sample_lenght = (im_lenght/coeff).astype(np.int)
indices = (np.random.rand(sample_lenght) * im_lenght).astype(np.int)

In [ ]:
def get_value_from_data_info(key, data_info, index):
    return data_info[key][index[0]]
    
    
def get_params_from_data_info(data_info, index):

    porosity = get_value_from_data_info('porosity', data_info, index)
    blobiness = get_value_from_data_info('blobiness', data_info, index)
    noise = get_value_from_data_info('noise', data_info, index)

    return porosity, blobiness, noise


def scatter_plot(x, y, colors, title):
    plt.figure(figsize=(10, 10))
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.scatter(x, y, marker='.', c=colors)
    plt.title(title)


def scatter_plot_values(x, y, origin, title):
    x_part = np.take(x, indices)
    y_part = np.take(y, indices)
    origin_part = np.take(origin, indices)
    colors = ['red' if el else 'blue' for el in origin_part]
    scatter_plot(x_part, y_part, colors, title)


def otsu_bin(image):
    threshold = filters.threshold_otsu(image)
    return image > threshold, threshold


def li_bin(image):
    threshold = filters.threshold_li(image)
    return image > threshold, threshold


def local_bin(image):
    
    def l_b(im):
        return im > filters.threshold_local(im, 3, method='mean')
    
    if len(image.shape) == 2:
        return l_b(image)
    elif len(image.shape) == 3:
        return np.asarray([l_b(image[i, :, :]) for i in range(image.shape[0])])
    else:
        raise ValueError('incorrect image shape')


def niblack_bin(image):
    return image > filters.threshold_niblack(image, 3)


def binarize_image(image, orig_image, pp, npa):

    title = 'otsu binarization'
    otsu_bin_image, otsu_threshold = otsu_bin(image)
    scatter_plot_values(pp, npa, otsu_bin_image.flatten(), title)
    plt.axvline(x=otsu_threshold, color='gray')
    preview_volumes(orig_image, otsu_bin_image, title)
    print(f'{ title } jaccard score: { jaccard_score(otsu_bin_image.flatten(), orig_image.flatten()) }')
    
    title = 'li binarization'
    li_bin_image, li_threshold = li_bin(image)
    scatter_plot_values(pp, npa, li_bin_image.flatten(), title)
    plt.axvline(x=li_threshold, color='gray')
    preview_volumes(orig_image, li_bin_image, title)
    print(f'{ title } jaccard score: { jaccard_score(li_bin_image.flatten(), orig_image.flatten()) }')

    title = 'local binarization'
    local_bin_image = local_bin(image)
    scatter_plot_values(pp, npa, local_bin_image.flatten(), title)
#     preview_volumes(orig_image, local_bin_image, title)
    print(f'{ title } jaccard score: { jaccard_score(local_bin_image.flatten(), orig_image.flatten()) }')

    title = 'niblack binarization'
    niblack_bin_image = niblack_bin(image)
    scatter_plot_values(pp, npa, niblack_bin_image.flatten(), title)
#     preview_volumes(orig_image, niblack_bin_image, title)
    print(f'{ title } jaccard score: { jaccard_score(niblack_bin_image.flatten(), orig_image.flatten()) }')


def post_filter_handler(image, orig_image, filtered_image, title):
    
    preview_volumes(image, filtered_image, title)
    
    fi = filtered_image.flatten()
    npa = pc.count_neighbor_average_array(filtered_image).flatten()
    oi = orig_image.flatten()
    
    scatter_plot_values(fi, npa, oi, title)
    
    title = f'{ title } binarized'
    otsu_bin_image, otsu_threshold = otsu_bin(filtered_image)
    scatter_plot_values(fi, npa, otsu_bin_image.flatten(), title)
    plt.axvline(x=otsu_threshold, color='gray')
    preview_volumes(orig_image, otsu_bin_image, title)
    print(f'{ title } jaccard score: { jaccard_score(otsu_bin_image.flatten(), orig_image.flatten()) }')


def median_filter(image, orig_image):
    
    title = 'median filter'
    filtered_image = filters.median(image)
    post_filter_handler(image, orig_image, filtered_image, title)


def anisotropic_diffusion_filter(image, orig_image):

    title = 'anisotropic diffusion'
    filtered_image = smoothing.anisotropic_diffusion(image)
    post_filter_handler(image, orig_image, filtered_image, title)


def filter_image(image, orig_image):
    median_filter(image, orig_image)
    anisotropic_diffusion_filter(image, orig_image)
    

def count_npa_vs_pp(images_tag, preview=True):

    df = data_info[data_info['tag'] == images_tag]

    for index, data_id in np.ndenumerate(df['id_indx']):
        
        title = f'porosity { porosity }, blobiness { blobiness }, noise { noise }'

        # pp — proc_phantom
        # npa — neighbor_pixel_average
        # op — orig_phantom

        pp, npa, op, proc_phantom, orig_phantom = pc.count_neighbor_average_array_and_save(dim, data_id, images_tag)

        if preview:
            scatter_plot_values(pp, npa, op, title)
            binarize_image(proc_phantom, orig_phantom, pp, npa)
            filter_image(proc_phantom, orig_phantom)

In [ ]:
count_npa_vs_pp('train', preview=False)
count_npa_vs_pp('test')

In [ ]:
test_data_index = '001'
test_data = dm.get_data(dimension=dim, id_indx=test_data_index, what_to_return='csv', tag='test')
if test_data.empty:
    raise ValueError(f'test data is empty')

print(f'test data summary')
print(f'shape { test_data.shape }')
print(test_data)

title = f'porosity { porosity }, blobiness { blobiness }, noise { noise }'


@time_measurement
def train_and_test():

    train_data = data_info.query('tag == "train"')
    train_dataframe = pd.DataFrame()
    
    for index, data_id in np.ndenumerate(train_data['id_indx']):

        print(f'train index { data_id }')
        train_datum = train_data.query(f'id_indx == "{ data_id }"')

        if train_datum.empty:
            print(f'can\'t train model by data at index { data_id }')
            continue

        train_pixels_df = dm.get_data(dimension=dim, id_indx=data_id, what_to_return='csv', tag='train')
        print(f'shape { train_pixels_df.shape }')
        print(train_pixels_df)
        
        if train_dataframe.empty:
            train_dataframe = train_pixels_df
        else:
            train_dataframe = train_dataframe.append(train_pixels_df, ignore_index=True)
    
    print(f'train summary')
    print(f'shape { train_dataframe.shape }')
    print(train_dataframe)
    print('\n')
                
    print(f'train: { title }')
    X_train = np.asarray(train_dataframe[['neighbor_average', 'proc_phantom_pixel_values']])
    Y_train = np.asarray(train_dataframe['pixel_real_value'])
    train_scaler = preprocessing.StandardScaler()
    train_scaler.fit(X_train)
    train_scaler.transform(X_train)
    LR = LogisticRegression(C=1, solver='liblinear').fit(X_train, Y_train)

    print(f'mean: { train_scaler.mean_ }, var: { train_scaler.var_ }, samples_seen: { train_scaler.n_samples_seen_ }\n')

    print('test:')        
    X_test = np.asarray(test_data[['neighbor_average', 'proc_phantom_pixel_values']])
    Y_test = np.asarray(test_data['pixel_real_value'])
    test_scaler = preprocessing.StandardScaler()
    test_scaler.fit(X_test)
    test_scaler.transform(X_test)

    print(f'mean: { test_scaler.mean_ }, var: { test_scaler.var_ }, samples_seen: { test_scaler.n_samples_seen_ }\n')

    Y_predict = LR.predict(X_test)
    print(LR.coef_, LR.intercept_, LR.classes_)
    print(f'classification_report { classification_report(Y_test, Y_predict) }')
    print(f'prediction score: { LR.score(X_test, Y_test) }')
    print(f'jaccard score: { jaccard_score(Y_predict, Y_test) }')
    print(f'\n')

    coef = LR.coef_
    intercept = LR.intercept_
    x = X_test[:, 1]
    y = X_test[:, 0]

    def line(x0):
        return (-(x0 * coef[0, 1]) - intercept[0]) / coef[0, 0]

    xmin, xmax = x.min(), x.max()

    scatter_plot_values(x, y, Y_test, title)
    plt.plot([xmin, xmax], [line(xmin), line(xmax)], color='gray')

    scatter_plot_values(x, y, Y_predict, title)

    origin_image = np.reshape(Y_test, shape)
    predict_image = np.reshape(Y_predict, shape)

    preview_volumes(origin_image, predict_image, f'origin vs predicted { title }')


train_and_test()